In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import os
from PIL import Image
import pytesseract as tess
from pytesseract import image_to_string
import cv2,sys
import numpy as np
import re
from pynput.keyboard import Key,Controller

In [ ]:
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
def directoryCheck(state):
    #creates the folder for a state and district
    path="/Users/HP/Desktop"
    fpath=path+'/'+str(state)
    if os.path.exists(fpath):
        pass
    else:
        os.mkdir(path+'/'+state)

In [ ]:
def pdf_replace(addr,i):
    try:
        if i==0:
            os.rename('/Users/HP/Desktop/display_pdf', addr+'/doc_'+str(i)+'.pdf')
        else:
            os.rename('/Users/HP/Desktop/display_pdf('+str(i)+').pdf', addr+'/doc_'+str(i)+'.pdf')
    except:
        pass

In [ ]:
def restore_webpage(driver,state,dist):
    #after closing the browser it restores the webpage to previous configuration
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="sess_state_code_chosen"]').click()
    path='//*[@id="sess_state_code_chosen"]/div/div/input'
    stateForm=driver.find_element_by_xpath(path)
    stateForm.clear()
    stateForm.send_keys(state.text)
    stateForm.send_keys(u'\ue007')
    time.sleep(2)

    driver.find_element_by_xpath('//*[@id="sess_dist_code_chosen"]').click()
    path='//*[@id="sess_dist_code_chosen"]/div/div/input'
    distForm=driver.find_element_by_xpath(path)
    distForm.clear()
    distForm.send_keys(dist.text)
    distForm.send_keys(u'\ue007')
    time.sleep(1)

In [ ]:
def crop_image(driver):
    #captures screenshot and edits the image till the captcha
    driver.save_screenshot('securimage_show.png')
    img=Image.open("securimage_show.png")
    area=(306,514,449,562)
    img.crop(area).save("securimage_show.png")
    

In [ ]:
def process_image():
    #reads the image and thresholds it to binary image and removing noise
    img=cv2.imread("securimage_show.png")
    rows,cols,t=img.shape
    
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    for i in range(rows):
        for j in range(cols):
            if img[i,j]>=90:
                img[i,j]=255
    ret,img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)

    cv2.imwrite("securimage_show.png",img)


In [ ]:
# chrome_profile = webdriver.ChromeOptions()

# profile = {"download.default_directory": "C:/Users/HP/Desktop/",
#            "download.directory_upgrade": True,
#            "plugins.plugins_disabled": ["Chrome PDF Viewer"],
#           "safebrowsing_enabled":"False"}

# chrome_profile.add_experimental_option("prefs", profile)


In [ ]:
def download(url,fromDate,toDate):
    #creating a selenium webdriver which visits the url given as a parametre
    driver=webdriver.Chrome(executable_path='/Users/HP/Desktop/chromedriver')
    #preference(driver)
    driver.get(url)
    
    #clicks on court orders
    driver.find_element_by_xpath('//*[@id="leftPaneMenuCO"]').click()
    driver.switch_to.alert.accept()
    
    time.sleep(2)
    
    #clicks on select state
    driver.find_element_by_xpath('//*[@id="sess_state_code_chosen"]').click()
    
    #parsing html page using Beautiful soup and finding all states present
    htmlPage=driver.page_source
    soup=BeautifulSoup(htmlPage)
    stateData=soup.findAll('li',attrs={'class':'active-result'})
    
    #for loop iterarting over all states
    for state in stateData[1:6]:
        
        print(state)
        #selecting a state
        path='//*[@id="sess_state_code_chosen"]/div/div/input'
        stateForm=driver.find_element_by_xpath(path)
        stateForm.clear()
        
        stateForm.send_keys(state.text)#)
        stateForm.send_keys(u'\ue007')
        time.sleep(2)
        
        directoryCheck(state.text)
        
        #clicking on district 
        driver.find_element_by_xpath('//*[@id="sess_dist_code_chosen"]').click()
        
        #parsing html page using Beautiful soup and finding all districts present
        htmlPage=driver.page_source
        soup=BeautifulSoup(htmlPage)
        distData=[]
        distData=soup.findAll('li',attrs={'class':'active-result'})
        
        #for loop iterarting over all districts

        #print(distData)
        for dist in distData[2:6]:
            #selecting a district
            path='//*[@id="sess_dist_code_chosen"]/div/div/input'
            distForm=driver.find_element_by_xpath(path)
            distForm.clear()
            distForm.send_keys(dist.text)
            distForm.send_keys(u'\ue007')
            time.sleep(1)
            #print(distData)
            directoryCheck(state.text+'/'+dist.text)
            
            #clicking on court complexity 
            driver.find_element_by_xpath('//*[@id="court_complex_code_chosen"]').click()
                
            #parsing html page using Beautiful soup and finding all courts present
            htmlPage=driver.page_source
            soup=BeautifulSoup(htmlPage)
            courtData=[]
            courtData=soup.findAll('li',attrs={'class':'active-result'})
            
            #for loop iterarting over all courts
            for court in courtData[3:6]:
                #selecting a court
                driver.execute_script('window.scrollBy(0,-5000)')
                path='//*[@id="court_complex_code_chosen"]/div/div/input'
                courtForm=driver.find_element_by_xpath(path)
                courtForm.clear()
                courtForm.send_keys(court.text)
                courtForm.send_keys(u'\ue007')
                time.sleep(1)
                #print(courtData)
                directoryCheck(state.text+'/'+dist.text+'/'+court.text)
                
                #clicking on orderdate
                driver.find_element_by_xpath('//*[@id="COorderDate"]').click()
                
                #choosing From Date
                fpath='//*[@id="from_date"]'
                fDate=driver.find_element_by_xpath(fpath)
                fDate.clear()
                fDate.send_keys(fromDate)
                
                #choosing To Date
                tpath='//*[@id="to_date"]'
                tDate=driver.find_element_by_xpath(tpath)
                tDate.clear()
                tDate.send_keys(toDate)
                tDate.send_keys(u'\ue007')
                #while loop to by pass captcha
                while(1):
                    #crops image and binarizes image and extracts content from it
                    crop_image(driver)

                    process_image()
                    captcha_text=image_to_string(Image.open("securimage_show.png"))


                    #checking extracted text is null string or alphanumeric
                    if captcha_text=="" or not captcha_text.isalnum():
                        captcha_text="test123"
                                        
                    
                    #filling the captcha text
                    cpath='//*[@id="captcha"]'
                    cBox=driver.find_element_by_xpath(cpath)
                    cBox.clear()
                    cBox.send_keys(captcha_text)
                
                    #clicking on GO button
                    driver.find_element_by_xpath('//*[@id="goResetDiv"]/input[1]').click()
                    
                    # try and except block to find loop breaking conditions
                    try:
                        driver.switch_to.alert.accept()
                        driver.find_element_by_xpath('//*[@id="captcha_container_2"]/div[1]/a').click()
                    except:
                        time.sleep(2)
                        
                        htmlPage=driver.page_source
                        soup=BeautifulSoup(htmlPage)
                        check=soup.findAll('div',attrs={'style':'position: absolute; background: rgba(0, 0, 0, 0.5); width: 100%; height: 100%; z-index: 999; top: 0px; cursor: wait;'})
                        if len(check)==1:
                            break
                        htmlPage=driver.page_source
                        soup=BeautifulSoup(htmlPage)
                        check=soup.findAll('div',attrs={'id':'showList2','style':''})
                        if len(check)>0:
                            break
                        htmlPage=driver.page_source
                        soup=BeautifulSoup(htmlPage)
                        check=soup.findAll('div',attrs={'id':'errSpan','style':''})
                        if len(check)>0:
                            break

                print("happy")
                time.sleep(2)
                htmlPage=driver.page_source
                soup=BeautifulSoup(htmlPage)
                check=soup.findAll('div',attrs={'style':'position: absolute; background: rgba(0, 0, 0, 0.5); width: 100%; height: 100%; z-index: 999; top: 0px; cursor: wait;'})

                #else find all the links obtained and download them
                htmlPage=driver.page_source
                soup=BeautifulSoup(htmlPage)
                links=driver.find_elements_by_xpath('//*[@id="orderid"]')
                i=0
                for link in links:
                    link.click()
                    driver.switch_to.window(driver.window_handles[0])
                    i=i+1
                time.sleep(5)
                for j in range(i):
                    pdf_replace('/Users/HP/Desktop/'+state.text+'/'+dist.text+'/'+court.text,j)
                    print(state.text+'/'+dist.text+'/'+court.text)

In [ ]:
def main():
    #print "Date in form of dd/mm/yyyy"
    #fromDate=raw_input()
    #toDate=raw_input()
    url="http://services.ecourts.gov.in/ecourtindia_v6/"
    #download(url,fromDate,toDate)
    download(url,'06-12-2019','15-01-2020')

In [ ]:
if __name__=="__main__":
    main()

In [ ]:
# from PIL import Image
# import pytesseract

# im = Image.open("securimage_show.png")

# text = pytesseract.image_to_string(im, lang = 'eng')

# print(text)